In [ ]:
from random import randrange


EMPTY = " "

class Board:
    """
    Board for playing tic tac toe
    """

    def __init__(self):
        """
        Initial method
        """
        self._board = self.__reset_board()

    @staticmethod
    def __reset_board():
        return {int(idx): EMPTY for idx in range(1, 10)}

    def reset_board(self):
        self._board = self.__reset_board()

    def pos(self, pos):
        return self._board[pos]

    def play(self, player, pos):
        try:
            if self._board[pos] != EMPTY:
                print("Move already made. Play another position.")
                return
            self._board[pos] = player
        except KeyError:
            print("Invalid Position. Play from 1 to 9")

    def copy(self):
        """
        Return a copy of the board 
        """
        _copied = Board()
        _copied._board = self._board.copy()
        return _copied

    def get_available_moves(self):
        """
        Return available moves
        """
        return [idx for idx, value in self._board.items() if value == EMPTY]

    def __repr__(self):
        board = self._board
        board_str = f"{board[7]} | {board[8]} | {board[9]} \n"
        board_str += "--+---+--\n"
        board_str += f"{board[4]} | {board[5]} | {board[6]}\n"
        board_str += "--+---+--\n"
        board_str += f"{board[1]} | {board[2]} | {board[3]}\n"
        return board_str


class Player:
    """
    Class for representing players
    """
    def __init__(self, symbol, **kwargs):
        self.symbol = symbol

    def move(self, board):
        raise NotImplemented
    
    def __repr__(self):
        return self.symbol

class HumanPlayer(Player):
    def move(self, board):
        while True:
            pos = int(input("What's your move? "))
            if pos in board.get_available_moves():
                break
            print("Play Again...")
        board.play(self.symbol, pos)

class RandomPlayer(Player):
    def move(self, board):
        available_positions = board.get_available_moves()
        pos_index = randrange(0, len(available_positions))
        pos = available_positions[pos_index]
        board.play(self.symbol, pos)

class MCPlayer(Player):
    def __init__(self, symbol, **kwargs):
        super().__init__(symbol, **kwargs)
        self.trials = kwargs.get('trials', 1000)

    def move(self, board):
        mc = MonteCarloSimulation(board, self.symbol, trials = self.trials)
        pos = mc.simulate()
        board.play(self.symbol, pos)


class Game:
    victories = [
        (1, 2, 3),
        (4, 5, 6),
        (7, 8, 9),
        (1, 5, 9),
        (7, 5, 3),
        (1, 4, 7),
        (2, 5, 8),
        (3, 6, 9)
    ]

    def __init__(self, x_player, o_player, board, next_move = None):
        self.board = board
        self.x_player = x_player
        self.o_player = o_player
        self.next_move = next_move
        self.winner = None
        if self.next_move is None:
            self.next_move = self.x_player 

    def get_next_player(self, turn):
        return self.o_player if turn == self.x_player else self.x_player

    def compute_winner(self):
        board = self.board
        for p1, p2, p3 in self.victories:
            if board.pos(p1) == board.pos(p2) == board.pos(p3) != EMPTY:
                return board.pos(p1)
        return None

    def play(self, verbose=True):
        turn = self.next_move
        while True:
            if verbose:
                print()
                print(f"Player {turn} is your turn")
            turn.move(self.board)
            if verbose:
                print(self.board)
            turn = self.get_next_player(turn)
            winner = self.compute_winner()
            if winner:
                if verbose:
                    print(f"{winner} WON!!!")
                self.winner = winner
                break
            if not self.board.get_available_moves():
                if verbose:
                    print("Deu velha!")
                break

    def __repr__(self):
        return f"{ self.board }"

class MonteCarloSimulation:
    """
    Class for simulating n trial games for getting best moves for 
    AI agent
    """
    def __init__(self, board, turn, trials=1000):
        """
        Class initialization
        """
        self.board = board
        self.turn = turn
        o_player = RandomPlayer('O')
        x_player = RandomPlayer('X')
        self.trials = trials
        next_player = o_player if turn == 'O' else x_player
        self.games = [Game(x_player, o_player, self.board.copy(), next_player)
                      for idx in range(trials)]

    def simulate(self):
        """
        Method for generating simulated games
        """
        available = self.board.get_available_moves()
        scores = {k: 0 for k in available}

        for game in self.games:
            game.play(verbose=False)
            coef = 1
            if game.winner != self.turn:
                coef = -1
            for pos in available:
                if game.board.pos(pos) == self.turn:
                    scores[pos] += coef
                elif game.board.pos(pos) != EMPTY:
                    scores[pos] -= coef
            # Compute scores
        return max(scores, key=lambda k: scores[k])


In [ ]:
board = Board()
x_player = MCPlayer('X', trials=10000)
o_player = HumanPlayer('O')
game = Game(x_player, o_player, board)
game.play()


Player X is your turn
  |   |   
--+---+--
  | X |  
--+---+--
  |   |  


Player O is your turn
What's your move? 5
Play Again...
What's your move? 3
  |   |   
--+---+--
  | X |  
--+---+--
  |   | O


Player X is your turn
  |   | X 
--+---+--
  | X |  
--+---+--
  |   | O


Player O is your turn
What's your move? 1
  |   | X 
--+---+--
  | X |  
--+---+--
O |   | O


Player X is your turn
  |   | X 
--+---+--
  | X |  
--+---+--
O | X | O


Player O is your turn


KeyboardInterrupt: ignored

In [ ]:
1
